In [0]:
import logging
import traceback
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

class DeltaTableHandler(logging.Handler):
    def __init__(self, table_name):
        super().__init__()
        self.table_name = table_name
        self.spark = SparkSession.builder.getOrCreate()

    def emit(self, record):
        # Safely retrieve notebook context information
        try:
            notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
        except Exception:
            notebook_path = "N/A"
        try:
            job_id = dbutils.notebook.entry_point.getDbutils().notebook().getContext().jobId().get()
        except Exception:
            job_id = "N/A"
        try:
            run_id = dbutils.notebook.entry_point.getDbutils().notebook().getContext().runId().get()
        except Exception:
            run_id = "N/A"
        try:
            user = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()
        except Exception:
            user = "N/A"

        # Format exception information if present
        if record.exc_info:
            exception_text = ''.join(traceback.format_exception(*record.exc_info))
        else:
            exception_text = None

        log_entry = {
            "timestamp": datetime.utcnow().isoformat(),
            "level": record.levelname,
            "message": record.getMessage(),
            "notebook_name": notebook_path,
            "job_id": job_id,
            "run_id": run_id,
            "user": user,
            "function_name": record.funcName,
            "line_number": record.lineno,
            "exception": exception_text
        }

        # Define the schema
        schema = StructType([
            StructField("timestamp", StringType(), True),
            StructField("level", StringType(), True),
            StructField("message", StringType(), True),
            StructField("notebook_name", StringType(), True),
            StructField("job_id", StringType(), True),
            StructField("run_id", StringType(), True),
            StructField("user", StringType(), True),
            StructField("function_name", StringType(), True),
            StructField("line_number", IntegerType(), True),
            StructField("exception", StringType(), True)
        ])

        df = self.spark.createDataFrame([log_entry], schema=schema)
        df.write.mode("append").format("delta").option("mergeSchema", "true").saveAsTable(self.table_name)

In [0]:
# Set up logger with the DeltaTableHandler
def setup_logger(log_level=logging.INFO, delta_table_name="spire_catalog.logging.logs"):
    """
    Configure logger with DeltaTableHandler.
    
    Parameters:
    - log_level: Logging level (default: INFO)
    - delta_table_name: Name of the Delta table for logs (default: spire_catalog.logging.logs)
    
    Returns:
    - Configured logger
    """
    logger = logging.getLogger(__name__)
    logger.setLevel(log_level)
    
    # Add DeltaTableHandler for writing logs to Delta table
    delta_handler = DeltaTableHandler(delta_table_name)
    delta_handler.setLevel(log_level)
    logger.addHandler(delta_handler)
    
    #Add console handler for immediate feedback
    console_handler = logging.StreamHandler()
    console_handler.setLevel(log_level)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    console_handler.setFormatter(formatter)
    logger.addHandler(console_handler)
    
    return logger